This notebook imports the fundamental objects of the streamm.buildingblocks module and goes through the functionality of each

Let's start with a methane molecule object 

In [1]:
import streamm.structures.buildingblock as bb

In [2]:
import math

Create a particle object with tag methane

In [3]:
mol_i = bb.Buildingblock('methane')

In [4]:
print(mol_i)

 methane


You can read in the .xyz file from the structures example or create a methane geometry using a molecular viewer such as Avogadro (https://avogadro.cc/)

In [5]:
mol_i.read_xyz()

In [6]:
mol_i.bonded_nblist = mol_i.guess_nblist(0,radii_buffer=1.25)

Check that all the particles have been read in

In [7]:
print mol_i.n_particles

5


Check that the neighbor list was set correctly 

In [8]:
print mol_i.bonded_nblist

 NBlist of 5 particle with 8 connections


Looks good, you should have the geometry of a methane molecule with a C-H bond length of 1.2 Angstroms 

We want to use the functionality of the buildingblock object to join two methanes together to create alkyl chains of any length

So let's set two of the hydrogens to be reactive sites (rsites). 

You can view the numerical order of the atoms in Avogadro by setting the label to "atom number," however, Avogadro labels atoms from 1 to N, while streamm uses 0 to N-1

We will choose the first two hydrogens and set their rsite variable to 'RH'. It does not matter what this identifier is, as long as the same identifier is passed to the attach() function later. Also, if the identifiers are not unique, the order in which it appears in the particles list will also be used.  

In [9]:
mol_i.particles[1].rsite = 'RH'

In [10]:
mol_i.particles[2].rsite = 'RH'

Now use the find_rsites() function to create the dictionary of lists to be used by the attach() function

In [11]:
mol_i.find_rsites()

In [12]:
print mol_i.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



Pass the molecule to the attach function and set the rsite id's and the list positions of the rsites

In [13]:
mol_j = bb.attach(mol_i,mol_i,'RH',0,'RH',1,tag='ethane')

Write the .xyz to file to be viewed with a molecular viewer. 

In [14]:
mol_j.write_xyz()

While the ethane molecule was generated, the hydrogens are eclipsed rather than staggered. 

We can avoid this by using the prepattach() function to orient the molecule and remove the reactive site

In [15]:
mol_k = mol_i.prepattach('RH',0,dir=-1,yangle=90.0)

Then apply a shift to set the bond length

In [16]:
CC_bl = mol_i.particles[0].bonded_radius*2.0
mol_k.shift_pos([CC_bl,0.0,0.0])

Then apply a rotation to set the conformation to staggered. Use a 180.0 degree rotation to place the reactive site in the correct orientation for subsequent attachments.  

In [17]:
angle_rad = 180.0*math.pi/180.0 
mol_k.rotate_yz(angle_rad)

In [18]:
mol_l = mol_i.prepattach('RH',1,dir=1)

In [19]:
mol_m = bb.attachprep(mol_k,mol_l)

In [20]:
mol_m.tag = 'ethane'

In [22]:
for pk,p in mol_m.particles.iteritems():
    print pk,p

0 atom[0] C (C)
1 atom[1] H (H)
2 atom[2] H (H)
3 atom[3] H (H)
4 atom[4] C (C)
5 atom[5] H (H)
6 atom[6] H (H)
7 atom[7] H (H)


In [23]:
print mol_m.bonded_nblist.list 
print mol_m.bonded_nblist.index 

[1, 2, 3, 4, 0, 0, 0, 0, 5, 6, 7, 4, 4, 4]
[0, 4, 5, 6, 7, 11, 12, 13, 14]


In [24]:
mol_m.write_xyz()

In [25]:
print mol_m.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



Attachments can also be done in a loop 

In [26]:
alkly_n = (12-1)/2 # Number of ethanes to add to get a dodecyl 

In [27]:
print alkly_n

5


In [28]:
mol_n = mol_m 

In [29]:
mol_n.find_rsites()

In [30]:
print mol_n.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



In [31]:
for i in range(alkly_n):
    mol_n = bb.attach(mol_n,mol_m,'RH',1,'RH',0)

In [32]:
mol_n.tag = 'dodecyl'

In [33]:
mol_n.write_xyz()

Oh, so alkyl!